# Train our Philosophical Document Embedding Model

In [1]:
import torch

import loader
import models
import utility


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

c:\Users\lucas\Desktop\UCSD\256\philosophical_oracle\philosophy_oracle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
filename_category_mapping, category_label_mapping = loader.load_labeling_mappings()
filename_label_mapping = filename_category_mapping
for key, value in filename_label_mapping.items():
    filename_label_mapping[key] = int(category_label_mapping[value])

In [3]:
loader_params = {
    "num_labels": len(category_label_mapping),
    "batch_size": 4,
    "chunk_size": 5000,
    "chunk_overlap": 100,
    "balance_multiplier": 2,
}

model_params = {
    "input_dim": 384,
    "hidden_dim": 64,
    "output_dim": loader_params['num_labels'],
    "dropout": 0.35
}
train_params = {
    "epochs": 10,
    "learning_rate": 1e-3
}

# train_splits = {
    # "source_texts": ["base_texts"],
    # "human_summaries": ["wikipedia", "philosophize_this_transcripts"],
    # "gpt_data": ["gpt_summaries", "gpt_philosophy_game"],
    # "all_human_texts": ["base_texts", "wikipedia", "philosophize_this_transcripts"],
    # "all_summaries": ["wikipedia", "philosophize_this_transcripts", "gpt_summaries", "gpt_philosophy_game"],
    # "all_sources": ["base_texts", "wikipedia", "philosophize_this_transcripts", "gpt_summaries", "gpt_philosophy_game"],
# }

train_folders = ["base_texts", "wikipedia", "philosophize_this_transcripts", "gpt_summaries", "gpt_philosophy_game"]
test_folders = ["eval_data"]
filename_category_mapping, category_label_mapping = loader.load_labeling_mappings()

train_dataloader = loader.get_dataloader(train_folders, loader_params, filename_label_mapping, balance_data=True, print_info=False)
test_dataloader = loader.get_dataloader(test_folders, loader_params, filename_label_mapping, balance_data=False, print_info=False)

model = models.Classifier(**model_params)
optim = torch.optim.Adam(model.parameters(), lr=train_params['learning_rate'])
model.to(device)

print(f"Epochs  || Train Loss | Train Acc || Test Loss | Test Acc")
for epoch in range(train_params['epochs']):
    train_loss = []
    num_correct, num_total = 0, 0
    for iter, (embeddings, labels) in enumerate(train_dataloader):
        embeddings, labels = embeddings.to(device), labels.to(device)        
        optim.zero_grad()
        logits = model(embeddings)
        
        loss, correct_preds = utility.compute_loss_preds(logits, labels)
        loss.backward()
        optim.step()

        num_correct += correct_preds
        num_total += labels.shape[0]
        train_loss.append(loss.cpu().item())

    eval_loss, eval_acc = utility.evaluate_model(model, test_dataloader, device)
    train_loss = torch.tensor(train_loss)
    print(f" {epoch+1:<2}/{train_params['epochs']:>3} ||    {train_loss.mean():.3f}   |   {num_correct/num_total:.3f}   ||   {eval_loss:.3f}   |  {eval_acc:.3f}  ")

model.to("cpu")
model.save_model()

Dataloader from ('['data\\base_texts', 'data\\wikipedia', 'data\\philosophize_this_transcripts', 'data\\gpt_summaries', 'data\\gpt_philosophy_game']') created with 157 batches in 7.5 seconds.
Dataloader from ('['data\\eval_data']') created with 11 batches in 1.2 seconds.
Epochs  || Train Loss | Train Acc || Test Loss | Test Acc
 1 / 10 ||    1.908   |   0.230   ||   1.944   |  0.214  
 2 / 10 ||    1.601   |   0.419   ||   1.874   |  0.238  
 3 / 10 ||    1.224   |   0.590   ||   1.835   |  0.310  
 4 / 10 ||    1.032   |   0.643   ||   1.805   |  0.381  
 5 / 10 ||    0.868   |   0.722   ||   1.860   |  0.405  
 6 / 10 ||    0.747   |   0.746   ||   1.763   |  0.381  
 7 / 10 ||    0.660   |   0.758   ||   1.806   |  0.452  
 8 / 10 ||    0.584   |   0.808   ||   1.783   |  0.405  
 9 / 10 ||    0.560   |   0.806   ||   1.826   |  0.452  
 10/ 10 ||    0.487   |   0.835   ||   1.929   |  0.405  
Model saved to model_weights\classifier_29k_11242024_010615.pth
